In [1]:
import os
import numpy as np
import cv2
import mediapipe as mp
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [2]:
import sys  
sys.path.insert(0, './')
from pose_media import mediapipe_pose

In [3]:
DATA_PATH = "./DATA/" 
if not os.path.exists(DATA_PATH):
    os.mkdir(DATA_PATH)   

In [4]:
mp = mediapipe_pose()
cTime,pTime = 0,0

In [14]:
import cv2
import time

# Initialize video capture
cap = cv2.VideoCapture(0)
file = input("name : ")
path = "./dataset/noact/"
if not os.path.exists(path):
    os.mkdir(path)   

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MP4V')  # Use 'MP4V' codec for better compatibility
out = cv2.VideoWriter(path+'{}.mp4'.format(file), fourcc, 20.0, (640, 480))

# Record for 15 seconds
start_time = time.time()
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        frame = cv2.resize(frame, (640, 480))
        out.write(frame)
        cv2.imshow('Video', frame)

        # Check if 15 seconds have elapsed
        elapsed_time = time.time() - start_time
        if elapsed_time > 11:
            break

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


In [15]:
sequence_length = 30
path = "./dataset/"
if not os.path.exists(path):
    os.mkdir(path)
actions = os.listdir(path)
for action in actions:
    if not os.path.exists(DATA_PATH+action):
        os.mkdir(DATA_PATH+action)
    video_list = os.listdir(path+action)
    no_sequences = 30//len(video_list)
    extra_sequences = 30%len(video_list)
    for video in range(len(video_list)):
        cd = os.path.join(path+ action+"/" + video_list[video])
        cap = cv2.VideoCapture(cd)
        with mp.mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            if video < extra_sequences:
                no_sequences += 1
                pre_sequences = video * no_sequences 
            elif video == extra_sequences:
                pre_sequences = video * (no_sequences +1)
            else:
                pre_sequences = extra_sequences * (no_sequences +1) + (video - extra_sequences)*no_sequences
            for sequence in range(no_sequences):
                if not os.path.exists(DATA_PATH+action+"/"+str(pre_sequences+sequence)):
                    os.mkdir(DATA_PATH+action+"/"+str(pre_sequences+sequence))
                for frame_num in range(sequence_length):
                        ref,frame = cap.read()
                        try:
                            image, results = mp.mediapipe_detection(frame, holistic)
                        except:
                            break
                            
                        mp.draw_styled_landmarks(image, results)
                        cTime = time.time()
                        fps = 1 / (cTime - pTime)
                        pTime = cTime
                        cv2.putText(image,"FPS:" +str(int(fps)),(10,100), cv2.FONT_HERSHEY_PLAIN, 2,(255,0,190),2,cv2.LINE_AA)

                        if frame_num == 0: 
                            cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, video), (5,15), 
                                              cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1, cv2.LINE_AA)
                            cv2.imshow('OpenCV Feed', image)
                        else: 
                            cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action,video), (5,15), 
                                            cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1, cv2.LINE_AA)
                            cv2.imshow('OpenCV Feed', image)

                        keypoints = mp.extract_keypoints(results)
                        npy_path = os.path.join(DATA_PATH, action, str(pre_sequences+sequence), str(frame_num))
                        np.save(npy_path, keypoints)
                        if cv2.waitKey(1) & 0xFF == ord("q"):
                            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
sequence_length = 30
path = "./dataset/"
actions = os.listdir(path)
section = 0

if not os.path.exists(DATA_PATH+actions[section]):
    os.mkdir(DATA_PATH+actions[section])
video_list = os.listdir(path+actions[section])
no_sequences = 100//len(video_list)
extra_sequences = 30%len(video_list)
for video in range(len(video_list)):
    cd = os.path.join(path+ actions[section]+"/" + video_list[video])
    cap = cv2.VideoCapture(cd)
    with mp.mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        if video < extra_sequences:
            no_sequences += 1
            pre_sequences = video * no_sequences 
        elif video == extra_sequences:
            pre_sequences = video * (no_sequences +1)
        else:
            pre_sequences = extra_sequences * (no_sequences +1) + (video - extra_sequences)*no_sequences
        for sequence in range(no_sequences):
            if not os.path.exists(DATA_PATH+actions[section]+"/"+str(pre_sequences+sequence)):
                os.mkdir(DATA_PATH+actions[section]+"/"+str(pre_sequences+sequence))
            for frame_num in range(sequence_length):
                ref,frame = cap.read()
                try:
                    image, results = mp.mediapipe_detection(frame, holistic)
                except:
                    break
                            
                mp.draw_styled_landmarks(image, results)
                cTime = time.time()
                fps = 1 / (cTime - pTime)
                pTime = cTime
                cv2.putText(image,"FPS:" +str(int(fps)),(10,100), cv2.FONT_HERSHEY_PLAIN, 2,(255,0,190),2,cv2.LINE_AA)

                if frame_num == 0: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(actions[section], video), (5,15), 
                                              cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(actions[section],video), (5,15), 
                                            cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)

                keypoints = mp.extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, actions[section], str(pre_sequences+sequence), str(frame_num))
                np.save(npy_path, keypoints)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
    cap.release()
    cv2.destroyAllWindows()

In [11]:
# DATA_PATH = os.mkdir('data_npy')
# DATA_PATH = os.path.join('data_npy')
# actions = np.array(["HANDCLAPPING","RUNNING","WALKING","STANDING"])
actions = np.array(["DRINKPIL", "NOACT"])
no_sequences = 50
sequence_length = 30

In [6]:
label_map = {label:num for num,label in enumerate(actions)}
label_map

{'DRINKPIL': 0, 'NOACT': 1}

In [ ]:
def load_data(actions,no_sequences,sequence_length):
    sequences,labels = [],[]
    for action in actions:
        for sequence in range(no_sequences):
            window = []
            for frame_num in range(sequence_length):
                res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
                window.append(res)
            sequences.append(window)
            labels.append(label_map[action])
    return sequences,labels

In [ ]:
def load_data(actions,sequence_length):
    path = "./DATA/"
    sequences,labels = [],[]
    for action in actions:
        dataset = os.listdir(path+action)
        for data in range(len(dataset)):
            window = []
            for frame_num in range(sequence_length):
                res = np.load(os.path.join(DATA_PATH,action,str(data),"{}.npy".format(frame_num)))
                window.append(res)
            sequences.append(window)
            labels.append(label_map[action])
    return sequences,labels

In [ ]:
# sequences,labels = [],[]
# for action in os.listdir("xxx/"):
#     fd = os.path.join("xxx/" + action)
#     window = []
#     for folder in os.listdir(fd):
#         fd1 = os.path.join("xxx/" + action +"/"+ folder)
#         for file in os.listdir(fd1):
#             res = np.load(os.path.join("xxx/" + action +"/"+ folder + "/" + file))
#             window.append(res)
#         sequences.append(window)
#         labels.append(label_map[action])

In [15]:
sequences,labels = load_data(actions,sequence_length)

TypeError: 'list' object cannot be interpreted as an integer

In [33]:
X = np.array(sequences)
X.shape

(40, 60, 258)

In [34]:
y = to_categorical(labels).astype(int)
y

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1]])

In [35]:
DATA_NPY = "./data_npy/" 
if not os.path.exists(DATA_NPY):
    os.mkdir(DATA_NPY) 
with open('data_npy/data_x.npy', 'wb') as f:
    np.save(f, X)
with open('data_npy/data_y.npy', 'wb') as f:
    np.save(f, y)